Aqui comenzare con Las funciones.

In [1]:
import pandas as pd
import numpy as np

cargo los datos.

In [2]:
df_reviews = pd.read_csv('reviews.csv')

In [3]:
df_items = pd.read_csv('items.csv')

In [4]:
df_steam_games = pd.read_csv('steam_games.csv')

Comenzamos con las funciones

Primera función:

In [5]:
def PlayTimeGenre(genre):
    juegos_genero_steam = df_steam_games[df_steam_games['genres'].str.lower().str.contains(genre)]
    juegos_genero_items = df_items[df_items['game_name'].isin(juegos_genero_steam['game_name'])]

    df_combinado = pd.merge(juegos_genero_items, juegos_genero_steam[['game_name', 'release_date']], on='game_name')

    playtime_por_año = df_combinado.groupby('release_date')['playtime_forever'].sum()

    return print('el año de lanzamiento con más horas jugadas para el genero:',genre,'es:',playtime_por_año.idxmax())

In [6]:
PlayTimeGenre('action')

el año de lanzamiento con más horas jugadas para el genero: action es: 2012


Segunda función:

In [7]:
def UserForGenre(genre):
    juegos_genero_steam = df_steam_games[df_steam_games['genres'].str.lower().str.contains(genre)]
    juegos_genero_items = df_items[df_items['game_name'].isin(juegos_genero_steam['game_name'])]

    df_combinado = pd.merge(juegos_genero_items, juegos_genero_steam[['game_name', 'release_date']], on='game_name')

    playtime_por_año_jugador = df_combinado.groupby(['user_id','release_date','playtime_forever'])['playtime_forever'].sum()

    pw = playtime_por_año_jugador.idxmax()

    return print(f"El usuario con más horas jugadas para Género{genre} es {pw[0]} con {pw[2]} horas jugadas en el año {pw[1]}")

In [8]:
UserForGenre('action')

El usuario con más horas jugadas para Géneroaction es Evilutional con 635295.0 horas jugadas en el año 2012


Tercera función:

In [9]:
def UsersRecommend(año):
    reviews_año = df_reviews[df_reviews['posted'] == año]

    re_año_recom = reviews_año[(reviews_año['recommend'] == 1) & (reviews_año['sentiment analysis'] >= 1)]

    juegos_recomendados = pd.merge(re_año_recom[['id_game','recommend']],df_steam_games[['id_game','game_name']],on='id_game')

    games_recom_sum = juegos_recomendados.groupby('game_name')['recommend'].sum()

    top_3 = games_recom_sum.nlargest(3).index

    return print(f'este es el top 3 juegos mas recomendados de steam: Puesto1: {top_3[0]}, Puesto2: {top_3[1]}, Puesto3: {top_3[2]}')

In [10]:
UsersRecommend(2015)

este es el top 3 juegos mas recomendados de steam: Puesto1: Counter-Strike: Global Offensive, Puesto2: Team Fortress 2, Puesto3: Garry's Mod


Cuarta función:

In [11]:
def UsersNotRecommend(año):
    reviews_año = df_reviews[df_reviews['posted'] == año]

    re_año_recom = reviews_año[(reviews_año['recommend'] == 0) & (reviews_año['sentiment analysis'] == 0)]

    juegos_recomendados = pd.merge(re_año_recom[['id_game','recommend']],df_steam_games[['id_game','game_name']],on='id_game')

    juegos_recomendados['recommend'] = juegos_recomendados['recommend'].replace({0:1})

    games_recom_sum = juegos_recomendados.groupby('game_name')['recommend'].sum()

    top_3 = games_recom_sum.nlargest(3).index

    print(f'este es el top 3 juegos menos recomendados de steam: Puesto1: {top_3[0]}, Puesto2: {top_3[1]}, Puesto3: {top_3[2]}')

In [12]:
UsersNotRecommend(2012)

este es el top 3 juegos menos recomendados de steam: Puesto1: Call of Duty®: Black Ops II, Puesto2: Red Faction®: Armageddon™, Puesto3: Resident Evil: Operation Raccoon City


Quinta función:

In [13]:
def sentiment_analysis(año):
    año_games = df_steam_games[df_steam_games['release_date'] == año]

    año_games_re = pd.merge(año_games[['game_name','id_game']],df_reviews[['id_game','sentiment analysis']],on='id_game')
    if año_games_re['sentiment analysis'].value_counts().sum() == 0:
        return print(f'El año {año} no tiene reseñas de ninguna clase')
    else:
        se = año_games_re['sentiment analysis'].value_counts()
        return print(f'Las reviews del año {año} son: Positivas: {se[2]}, Neutras:{se[1]}, Negativas: {se[0]}')


In [14]:
sentiment_analysis(2015)

Las reviews del año 2015 son: Positivas: 2823, Neutras:812, Negativas: 898


Sistema de recomendacion:

Selecionare las Columnas que usare en mi modelo de ML pero antes:

Con el dataframe de games hare un tratamiento especial, traere unas columnas mas y desanidare las columnas que sean lista de str:

In [15]:
import json

# Lista para almacenar los objetos JSON 
user_games = []

# Abre el archivo JSON y carga su contenido en un diccionario
with open('data/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        try:
            json_data = json.loads(line)
            user_games.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en línea: {e}")

# Crear el DataFrame anidado
df_steam_games = pd.DataFrame(user_games)

Eliminare las filas que no sirven

In [16]:
df_steam_games.drop(df_steam_games.index[0:88310],inplace=True)

Selecciono las columnas que estare usando

In [17]:
df_steam_games = df_steam_games[['genres','app_name','release_date','id','price','specs']]

Desanidare la columna de genres:

In [18]:
games = df_steam_games.explode('genres')

In [19]:
games.dropna(subset=['genres'],inplace=True)

In [20]:
games.dropna(inplace=True)

In [21]:
games.fillna({'genres': 'Unknown'},inplace=True)
games.drop_duplicates(subset='id',inplace=True)

Hare lo mismo con la columna de specs

In [22]:
games = games.explode('specs')

In [23]:
games = games.fillna({'specs': 'Unknown'})

In [24]:
games.drop_duplicates(subset='id',inplace=True)

Normalizare la fecha solo poniendo el año

In [25]:
games['release_date']=pd.to_datetime(games['release_date'],errors="coerce").dt.year

In [26]:
games.dropna(subset=['release_date'], inplace=True)

In [27]:
games['release_date'] = games['release_date'].astype(int).astype(str)

Reseteo el index

In [28]:
games.reset_index(inplace=True,drop=True)

Hare una estandarizacion y una clasificacion de la columna price

In [29]:
replace_dict = {
    'Free To Play': 0,
    'Free to Play': 0,
    'Free': 0,
    'Free Demo': 0,
    'Play for Free!': 0,
    'Install Now': 0,
    'Play WARMACHINE: Tactics Demo': 0,
    'Free Mod': 0,
    'Third-party': 0,
    'Play Now': 0,
    'Free HITMAN™ Holiday Pack': 0,
    'Play the Demo': 0,
    'Install Theme':0
}
games['price'].replace(replace_dict, inplace=True)

Luego de estandarizar la columna price, comenzaré con la clasificacion.

Obtengo la mediana de los precios y etiqueto en base a la mediana.

In [30]:
median_price = games['price'].median()
median_price

4.99

In [31]:
def etiquetar_precio(valor):
    if valor == 0:
        return 'gratis'
    elif valor <= 4.99:
        return "barato"
    else:
        return "caro"

In [32]:
games['price'] = games['price'].apply(etiquetar_precio)

Pasamos la estandarizacion a numeros.

In [33]:
price = {'gratis':0,'barato':1,'caro':2}

In [34]:
games['price'].replace(price,inplace=True)

Aqui cambiamos el nombre de la columna id a id_game.

In [35]:
games.rename({'id':'id_game'},axis=1,inplace=True)

Haré una clasificacion numerica a las columnas que estare mostrando para tener un mejor resultado en el modelo de recomendacion.

In [36]:
from sklearn.preprocessing import LabelEncoder
le_genres = LabelEncoder()
le_date = LabelEncoder()
le_specs = LabelEncoder()

In [37]:
games['genres'] = le_genres.fit_transform(games['genres'])
games['release_date'] = le_date.fit_transform(games['release_date'])
games['specs'] = le_specs.fit_transform(games['specs'])

Aqui comenzare con la funcion "item-item"

In [100]:
def item_item(id_game,games):
    #aquí copiare el dataframe games para utilizarlo en el modelo ML
    games_co = games.copy()

    #dropeamos las columnas:
    games_co.drop(['app_name','id_game'],axis=1,inplace=True)

    # importamos la funcion cosine_similarity, cargamos el resultado en games_coceno y convertimos en dataframe.
    from sklearn.metrics.pairwise import cosine_similarity
    games_co = cosine_similarity(games_co)
    games_co = pd.DataFrame(games_co)

    index = games[games['id_game'] == str(id_game)].index[0]
    recomend = games_co[index].sort_values(ascending=False).iloc[1:6].index
    recomend = games.loc[recomend, ['id_game','app_name']].values

    return(f'estos son los juegos recomendados con sus ids y nombres: 1 id:{int(recomend[0][0])} name: {recomend[0][1]}, 2 id:{int(recomend[1][0])} name: {recomend[1][1]}, 3 id:{int(recomend[2][0])} name: {recomend[2][1]}, 4 id:{int(recomend[3][0])} name: {recomend[3][1]}, 5 id:{int(recomend[4][0])} name: {recomend[4][1]}.')

Aqui creamos la función "user-item"

In [101]:
def user_item(user,items,games):
    # filtramos los usuarios para traer todos los juegos jugados por el.
    items = items[items['user_id'] == str(user)]
    #traeré el id del juego mas jugado por ese usuario.
    max = items['playtime_forever'].idxmax()
    id_game = items['id_game'].iloc[max]
    #Usaré el id del juego y lo metemos a la función que habia creado antes para encontrar los 5 juegos recomendados
    return item_item(id_game,games)

importo el archivo games para usarlo en las funciones.

In [103]:
games.to_csv('games_model.csv',index=False)